In [10]:
from flask import Flask, request
import json
import requests
import os

from langchain.chains import AnalyzeDocumentChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from variables import API_KEY_CHATGPT


In [ ]:
def read_file():
    try:
        path = f"{os.getcwd()}\\uploads"
        for file in os.listdir(path):
            python_file = file

        file_path = os.path.join(path, python_file)        
        with open(file_path, 'r') as file:
            file_content = file.read()
            print(file_content)
        answer = {'file': file_content}
    except FileNotFoundError:
        answer = { 'error' : f"El archivo '{file_path}' no fue encontrado."}
    except Exception as e:
        answer = { 'error':  f"Error al leer el archivo: {e}"}
        return answer

In [ ]:
read_file()

import json
from flask import Flask, request
import sqlite3
import pickle
import pandas as pd
import os

def clean_data(data):
    columns = data.columns
    if "Unnamed: 0" in columns:
        data.drop(columns='Unnamed: 0', inplace=True)

    index = data[data["newpaper"].str.contains(r".*[a-zA-Z].*")].index  
    data.loc[index,"newpaper"] = data.loc[index,"newpaper"].str.replace("s","")
    data['newpaper'] = data['newpaper'].astype(float)
    data.rename(columns={"newpaper" : "newspaper"}, inplace=True)

    return data

def connect_db():
    conn = sqlite3.connect('advertising_new.db')

    return conn

def close_db(conn):
    conn.close()

def create_db(conn):
    cursor = conn.cursor()

    create_table = \
    """
        CREATE TABLE IF NOT EXISTS advertising (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            TV FLOAT(32),
            radio FLOAT(32),
            newspaper FLOAT(32),
            sales FLOAT(32)
        )
    """
    cursor.execute(create_table)



In [6]:
from datetime import datetime

In [7]:
def chat_gpt_response():

    error = None
    answer = {}
    #question = request.json()
    question_test = "Cuentame que endpoints tiene el archivo"

    llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=API_KEY_CHATGPT)

    qa_chain = load_qa_chain(llm, chain_type="map_reduce")

    qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

    try:
        path = f"{os.getcwd()}\\uploads"
        for py_file in os.listdir(path):
            python_file = py_file

        file_path = os.path.join(path, python_file)        
        with open(file_path, 'r') as file_p:
            file = file_p.read()

        response = qa_document_chain.run(
            input_document=file,
            question=question_test,
        )
        time = datetime.now()
        answer = {
            'question': question_test,
            'response': response,
            'time': time.isoformat()
        }

    except FileNotFoundError:
        error = f"El archivo '{file_path}' no fue encontrado."
    except Exception as e:
        error = f"Error al leer el archivo: {e}"

    if error:
        answer = {'error': error}

    return answer

In [8]:
chat_gpt_response()

{'question': 'Cuentame que endpoints tiene el archivo',
 'response': 'El archivo tiene los siguientes endpoints:\n\n1. `/` : Este endpoint se utiliza para dar la bienvenida a la API y establecer la conexión con la base de datos.\n\n2. `/predict` : Este endpoint se utiliza para realizar predicciones utilizando un modelo de machine learning. Se puede enviar una solicitud GET para obtener las predicciones o se puede enviar una solicitud POST con datos de entrada en formato JSON para obtener predicciones personalizadas.\n\n3. `/ingest` : Este endpoint se utiliza para ingresar nuevos datos a la base de datos. Se debe enviar una solicitud POST con datos de entrada en formato JSON.\n\n4. `/retrain` : Este endpoint se utiliza para volver a entrenar el modelo de machine learning. Se debe enviar una solicitud POST.',
 'time': '2023-12-14T17:00:16.712806'}

In [9]:
time = datetime.now()
time.isoformat()

'2023-12-14T17:00:16.717630'